In [ ]:
import itertools
import pandas as pd
from itertools import combinations
from collections import defaultdict
import re

# Параметры, которые можно настроить:
csv_path = input("Введите путь к CSV-файлу CICIDS2017: ").strip()
min_support = 10  # минимальная поддержка (количество транзакций), примерное значение
num_bins = 5      # количество интервалов для дискретизации числовых признаков
chunk_size = 10000 # размер чанка при потоковой загрузке CSV

# Определяем названия столбцов, которые нужно дискретизировать (числовые признаки)
numeric_columns = ['Flow Duration' , 'Active Mean' ,'Idle Mean' , 'Average Packet Size', 'Total Fwd Packets', 'Total Length of Fwd Packets' ]
# Примечание: Список numeric_columns можно расширить или скорректировать под фактические столбцы набора данных.
# Определяем столбцы, которые являются уже категориальными (в том числе Label)
categorical_columns = ['FIN Flag Count','SYN Flag Count','RST Flag Count','PSH Flag Count','ACK Flag Count'] #'
# Примечание: "Protocol" может быть числовым кодом, но мы будем считать его категориальным (например, Protocol=6 как TCP).
# "Label" - метка атаки, уже категориальная.

# Словари для сбора глобального min и max по числовым столбцам
min_values = {col: float('inf') for col in numeric_columns}
max_values = {col: float('-inf') for col in numeric_columns}

# Первый проход: находим глобальные min и max для числовых признаков
for chunk in pd.read_csv(csv_path, chunksize=chunk_size):
    # Обновляем минимумы и максимумы
    for col in numeric_columns:
        if col in chunk.columns:
            col_min = chunk[col].min(skipna=True)
            col_max = chunk[col].max(skipna=True)
            # Обновляем глобальные значения
            if pd.notna(col_min):
                if col_min < min_values[col]:
                    min_values[col] = col_min
            if pd.notna(col_max):
                if col_max > max_values[col]:
                    max_values[col] = col_max

# Определяем интервалы (bins) для каждого числового столбца
bin_edges = {}
for col in numeric_columns:
    if min_values[col] == float('inf'):
        # Если столбца нет в данных или не найден, пропускаем
        continue
    # Добавляем небольшой эпсилон к max чтобы включить правый край в последний интервал
    min_val = min_values[col]
    max_val = max_values[col]
    if min_val >= max_val:
        # Если значение постоянно, можно создать одну категорию
        bin_edges[col] = [min_val]
    else:
        # Равномерные интервалы
        bin_edges[col] = [min_val + i * (max_val - min_val) / num_bins for i in range(1, num_bins)] 
        # (Будет num_bins-1 внутренних границ; pandas.cut сам добавит min и max)


In [ ]:
# Структуры для подсчета поддержки
item_support = defaultdict(int)  # частота каждого элемента
pair_support = defaultdict(int)  # частота каждой пары элементов

# Список транзакций (можем записывать во временный файл, но для простоты собираем список, 
# при очень больших данных стоит обрабатывать по частям)
transactions = []

for chunk in pd.read_csv(csv_path, chunksize=chunk_size):
    for _, row in chunk.iterrows():
        transaction_items = []
        # Преобразуем числовые признаки в категориальные по бинам
        for col in numeric_columns:
            if col in row and pd.notna(row[col]):
                val = row[col]
                # Находим бин (категорию) для значения
                if min_values[col] == float('inf'):
                    continue  # столбец отсутствует
                if min_values[col] >= max_values[col]:
                    # Константное значение, используем как есть
                    cat_label = f"{col}={val}"
                else:
                    # Определяем интервал через pd.cut (разовое использование с нашими границами)
                    bins = [min_values[col]] + bin_edges[col] + [max_values[col] + 1e-9]  # + эпсилон к последнему
                    # Найдём индекс интервала, в который попадает значение
                    # Пройдём по границам, чтобы определить категорию
                    cat_idx = None
                    for i in range(len(bins)-1):
                        if bins[i] <= val < bins[i+1]:
                            cat_idx = i
                            break
                    if cat_idx is None:
                        cat_idx = len(bins) - 2  # если значение == max_value
                    cat_label = f"{col}=Bin{cat_idx}"
                transaction_items.append(cat_label)
        # Обрабатываем категориальные признаки
        for col in categorical_columns:
            if col in row and pd.notna(row[col]):
                val = row[col]
                transaction_items.append(f"{col}={val}")
        # Обновляем поддержку для каждого элемента и каждой пары
        # Удаляем дубликаты в транзакции (если таковые есть), т.к. для алгоритма ассоциации важна только присутствие
        transaction_items = list(set(transaction_items))
        # Пропускаем пустые транзакции
        if not transaction_items:
            continue
        # Учитываем поддержку элементов
        for item in transaction_items:
            item_support[item] += 1
        # Учитываем поддержку пар элементов (совстречаемость)
        # Используем itertools.combinations для генерации всех пар (без учета порядка)
        for item_a, item_b in combinations(sorted(transaction_items), 2):
            pair_support[(item_a, item_b)] += 1
        # Сохраняем транзакцию во временный список
        transactions.append(transaction_items)

# Фильтрация редких элементов по порогу поддержки
frequent_items = {item for item, sup in item_support.items() if sup >= min_support}
# Если элемент не частый, удаляем его из всех транзакций и не учитываем в дальнейшем
filtered_transactions = []
for items in transactions:
    # Оставляем только частые элементы
    freq_items = [item for item in items if item in frequent_items]
    if freq_items:
        filtered_transactions.append(freq_items)
transactions = filtered_transactions  # заменяем исходные транзакции отфильтрованными

# Освобождаем память, удаляя вспомогательные структуры, если они больше не нужны
del filtered_transactions


In [ ]:
# Класс для узлов FP-дерева
class FPNode:
    __slots__ = ("item", "count", "parent", "children", "link", "array", "leaf")
    def __init__(self, item, parent):
        self.item = item       # Название элемента (None для корня)
        self.count = 1         # Инициализируем счетчик
        self.parent = parent   # Ссылка на родительский узел
        self.children = {}     # Словарь: элемент -> дочерний узел
        self.link = None       # Ссылка на следующий узел такого же элемента
        self.array = 0         # Маркер использования "array" (будет 1 для узлов разреженных элементов)
        self.leaf = True       # Лист (если появятся дети, переключим на False)

# Строим FP-дерево
header_table = {}  # Таблица заголовков: элемент -> первый узел в цепочке данного элемента
root = FPNode(None, None)  # корневой узел (item=None)

# Получаем порядок сортировки элементов по частоте (убывание) из частотной таблицы
item_order = sorted(item_support.keys(), key=lambda x: item_support[x], reverse=True)

for transaction in transactions:
    # Сортируем элементы транзакции по глобальной частоте (убывание)
    sorted_items = sorted(transaction, key=lambda x: item_support[x], reverse=True)
    # Вставляем по порядку в дерево
    current_node = root
    for item in sorted_items:
        # Если дочерний узел с таким item существует - увеличиваем счетчик
        if item in current_node.children:
            child = current_node.children[item]
            child.count += 1
        else:
            # Создаем новый узел
            child = FPNode(item, current_node)
            current_node.children[item] = child
            # Добавляем узел в таблицу заголовков
            if item in header_table:
                # Найдем последний узел в цепочке link и добавим новый
                last_node = header_table[item]
                while last_node.link is not None:
                    last_node = last_node.link
                last_node.link = child
            else:
                header_table[item] = child
        # При добавлении нового узла у родительского узла уже не лист
        current_node.leaf = False
        # Переходим на уровень ниже
        current_node = child

# После построения дерева, помечаем leaf для конечных узлов явно (для всех узлов, которые остались листами)
# (Уже учтено: мы устанавливали leaf=False при добавлении детей)

# Определяем порог для разреженных элементов (например, threshold = 5 узлов)
sparse_threshold = 5

# Вычисляем число узлов для каждого элемента по цепочке link
item_node_count = {}
for item, node in header_table.items():
    count = 0
    current = node
    while current is not None:
        count += 1
        current = current.link
    item_node_count[item] = count

# Отмечаем узлы: если элемент разреженный (узлов >= threshold), устанавливаем node.array = 1 для всех его узлов
sparse_items = {item for item, cnt in item_node_count.items() if cnt >= sparse_threshold}
dense_items = set(item_node_count.keys()) - sparse_items

for item in sparse_items:
    node = header_table.get(item)
    while node is not None:
        node.array = 1
        node = node.link


In [ ]:
# Максимальный размер наборов
max_itemset_size = 7

# Словарь для хранения всех частых наборов (ключ – frozenset набора, значение – поддержка)
frequent_itemsets = {}

# -------------------------------
# Шаг 1. Частые 1-элементные наборы
# -------------------------------
L1 = set()
for item, sup in item_support.items():
    if sup >= min_support:
        fs_item = frozenset([item])
        L1.add(fs_item)
        frequent_itemsets[fs_item] = sup

# -------------------------------
# Шаг 2. Частые 2-элементные наборы
# Используем sparse_items для генерации пар
# -------------------------------
L2 = set()
for item in sparse_items:
    for other, sup in item_support.items():
        if other == item:
            continue
        candidate = frozenset([item, other])
        # Упорядочим пару для проверки в pair_support
        pair = tuple(sorted(candidate))
        if pair in pair_support and pair_support[pair] >= min_support:
            L2.add(candidate)
            frequent_itemsets[candidate] = pair_support[pair]

# Сохраняем уровневые частые наборы в список
# levels[0] соответствует 1-элементным, levels[1] – 2-элементным наборам
levels = []
levels.append(L1)
levels.append(L2)

# -------------------------------
# Шаг 3. Генерация кандидатов для наборов размера 3 до max_itemset_size
# -------------------------------
for k in range(3, max_itemset_size + 1):
    prev_level = list(levels[-1])
    candidate_itemsets = set()
    
    # Объединяем два набора предыдущего уровня, если они имеют k-2 общих элементов
    for i in range(len(prev_level)):
        for j in range(i + 1, len(prev_level)):
            candidate = prev_level[i] | prev_level[j]
            if len(candidate) == k:
                # Проверка Apriori: все (k-1)-поднаборы кандидата должны быть частыми
                all_subsets_frequent = True
                for subset in itertools.combinations(candidate, k - 1):
                    if frozenset(subset) not in levels[-1]:
                        all_subsets_frequent = False
                        break
                if all_subsets_frequent:
                    candidate_itemsets.add(candidate)
                    
    # Подсчёт поддержки кандидатов, используя список транзакций
    current_level = set()
    for candidate in candidate_itemsets:
        support_count = 0
        for trans in transactions:
            if candidate.issubset(trans):
                support_count += 1
        if support_count >= min_support:
            current_level.add(candidate)
            frequent_itemsets[candidate] = support_count
            
    # Если не найдено ни одного частого набора, прекращаем дальнейшее расширение
    if not current_level:
        break
    levels.append(current_level)

# Теперь frequent_itemsets содержит все частые наборы от 1 до 7 элементов (или меньше, если на каком-то уровне наборов не нашлось)

In [ ]:
# Рекурсивный обход FP-дерева для извлечения частых наборов плотных элементов
frequent_paths = []  # для хранения найденных путей (наборов) с их поддержкой

def dfs(node, current_path):
    # Обходим каждого дочернего узла
    for item, child in node.children.items():
        # Если встречаем узел разреженного элемента, пропускаем его ветвь
        if child.array == 1:
            continue  # пропускаем все подветви этого узла (как указано в алгоритме для плотной обработки)
        # Добавляем элемент узла в текущий путь
        new_path = current_path + [child.item]
        # Если узел является листом или далее по ветви только разреженные, фиксируем путь
        if child.leaf or not any(grandchild.array == 0 for grandchild in child.children.values()):
            # Вычисляем поддержку для набора new_path.
            # Поддержка набора элементов = счетчик этого узла (так как в FP-дереве count узла = количество транзакций,
            # содержащих путь от корня до данного узла).
            support_count = child.count
            frequent_paths.append((frozenset(new_path), support_count))
        # Рекурсивно спускаемся глубже
        dfs(child, new_path)
    # (Возврат обратно не требуется, так как мы передаём копии списка current_path при углублении)

# Запускаем DFS от корня (корень не содержит элемента)
dfs(root, [])

# Добавляем все полученные частые наборы плотных элементов в словарь frequent_itemsets
for itemset, sup in frequent_paths:
    # Если набор уже есть (возможно, он найден и другим методом) и поддержка совпадает, можно пропустить, 
    # но на всякий случай мы можем обновить на наибольшую поддержку.
    if itemset in frequent_itemsets:
        existing_sup = frequent_itemsets[itemset]
        if sup > existing_sup:
            frequent_itemsets[itemset] = sup
    else:
        if sup >= min_support:
            frequent_itemsets[itemset] = sup


In [ ]:
import csv

# Группируем частые наборы по их размеру (длине)
sd_structure = defaultdict(list)
for itemset in frequent_itemsets.keys():
    length = len(itemset)
    sd_structure[length].append(itemset)

# Сортируем ключи (размеры) по возрастанию
sorted_lengths = sorted(sd_structure.keys())

# Подготавливаем данные для записи: превращаем каждый набор в строку
sd_rows = []
for length in sorted_lengths:
    patterns = sd_structure[length]
    # Преобразуем каждый набор во строковой формат, например "item1 & item2 & item3"
    pattern_strings = []
    for itemset in patterns:
        items = sorted(itemset)
        pattern_strings.append("{" + ", ".join(items) + "}")
    # Объединяем все паттерны данной длины в одну строку разделённую '; '
    patterns_joined = "; ".join(pattern_strings)
    sd_rows.append([length, patterns_joined])

# Сохраняем структуру SD в CSV файл
with open("sd_structure.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Length", "Patterns"])
    writer.writerows(sd_rows)


In [ ]:
# Параметры для правил
min_confidence = 0.9
min_lift = 1.5
num_transactions = len(transactions)

rules = []
for itemset, sup in frequent_itemsets.items():
    if len(itemset) < 2:
        continue  # пропускаем одноэлементные наборы, из них правил не сделать
    # Рассматриваем все разбиения itemset на A -> B
    # Для избежания дублирования, можем например перебирать все непустые A, где A не больше половины набора (но тогда и B будет повторяться).
    # Проще: перебирать все непустые подмножества A (кроме полного набора), тогда B определяется однозначно.
    items = list(itemset)
    n = len(items)
    # Перебираем размеры A от 1 до n-1
    for r in range(1, n):
        from itertools import combinations
        for combo in combinations(items, r):
            A = set(combo)
            B = itemset - A
            if not B:
                continue
            # Поддержка A и B
            support_A = frequent_itemsets.get(frozenset(A), None)
            support_B = frequent_itemsets.get(frozenset(B), None)
            # Если по каким-то причинам нет поддержки A или B в словаре (например, B не встречался как частый сам по себе),
            # то вычислим вручную, пробежав по транзакциям. Но чаще всего frequent_itemsets будет содержать их, особенно A, 
            # так как A может быть не частым по min_support, однако правило тогда не пройдет по confidence.
            if support_A is None:
                # Подсчет поддержки A путем прохода (медленно, но редко нужно)
                support_A = 0
                for trans in transactions:
                    if A.issubset(trans):
                        support_A += 1
            if support_B is None:
                support_B = 0
                for trans in transactions:
                    if B.issubset(trans):
                        support_B += 1
            # Вычисляем confidence и lift
            conf = sup / support_A if support_A > 0 else 0
            lift = (conf / (support_B / num_transactions)) if support_B and num_transactions else float('inf')
            if conf >= min_confidence and lift >= min_lift:
                rules.append((A, B, sup, conf, lift))

# Сохраняем правила в CSV
with open("association_rules.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Antecedent", "Consequent", "Support", "Confidence", "Lift"])
    for A, B, sup, conf, lift in rules:
        A_str = "{" + ", ".join(sorted(A)) + "}"
        B_str = "{" + ", ".join(sorted(B)) + "}"
        writer.writerow([A_str, B_str, sup, f"{conf:.2f}", f"{lift:.2f}"])


In [ ]:
# Подготовка данных для frequent_itemsets.csv
freq_rows = []
for itemset, sup in frequent_itemsets.items():
    items = sorted(itemset)
    itemset_str = "{" + ", ".join(items) + "}"
    freq_rows.append((len(itemset), itemset_str, sup))

# Отсортируем по размеру набора, затем по поддержке (убывание) например
freq_rows.sort(key=lambda x: (x[0], -x[2], x[1]))

# Запись в CSV
with open("frequent_itemsets.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Itemset", "Support"])
    for _, itemset_str, sup in freq_rows:
        writer.writerow([itemset_str, sup])
